# Importación de los Datos

In [31]:
# Importamos librerias y definimos parámetros
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

pd.options.mode.copy_on_write = True
pd.set_option('display.max_rows', 10)

In [32]:
# Leer el archivo excel en su totalidad
excel_file = 'C:/Información/Master Big Data n Analytics/TFM/2022-2023 Datos_solo_accesos.xlsx'
excel_completo = pd.read_excel(excel_file, sheet_name=None)

In [33]:
# Crear un diccionario para almacenar cada pestaña en una variable diferente
DF = {}

# Recorrer todas las pestañas y almacenarlas en el diccionario
for nombre_pestaña, df in excel_completo.items():
    DF[nombre_pestaña] = df

# Manipulación de datos

# Tabla de Accesos

In [34]:
# Vinculamos el DataFrame de Tabla de Accesos a la variable accesos
accesos = DF['C1329 Tabla_accesos']

In [35]:
# Rellenamos todas las celdas vacías con 0 para poder operar con el modelo en el futuro
accesos.replace(' ', np.nan, inplace = True)
accesos.fillna(0, inplace = True)

C:\Users\Joan\AppData\Local\Temp\ipykernel_3312\2738407397.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  accesos.replace(' ', np.nan, inplace = True)


In [36]:
# Convertimos los nombres de las columnas menusales a formato fecha para poder operar con las mismas
nombres_columna_mes = accesos.columns[1:]
mes_fecha = datetime(2022, 1, 1)

for mes in nombres_columna_mes:
    accesos.rename(columns={mes: mes_fecha}, inplace=True)
    mes_fecha = pd.Timestamp(mes_fecha) + pd.DateOffset(months=1)

In [37]:
# Pasamos los meses de todas las columnas a una única columna con los 24 meses para cada IdPersona
accesos = pd.melt(accesos, id_vars=['IdPersona'], var_name='Mes', value_name='Accesos')
accesos = accesos.sort_values(by=['IdPersona', 'Mes'])

In [38]:
# Añadimos 2 columnas de lags y eliminamos las filas con valores nulos
accesos['Accesos-1'] = accesos.groupby('IdPersona')['Accesos'].shift(1)
accesos['Accesos-2'] = accesos.groupby('IdPersona')['Accesos'].shift(2)
accesos.dropna(inplace = True)

In [39]:
# Dataset reducido para realizar pruebas:

# usuarios = [15, 20, 39, 56, 126, 149, 151, 161, 34562, 55546]
# accesos = accesos[accesos['IdPersona'].isin(usuarios)]

# Creación tabla información de cliente y tablas de variables menusales

In [40]:
# Juntamos las 24 tablas mensuales y creamos un DataFrame
dfs_mensuales = [DF['Ene_22'], DF['Feb_22'], DF['Mar_22'], DF['Abr_22'], DF['May_22'], DF['Jun_22'], DF['Jul_22'], DF['Ago_22'], DF['Sep_22'], DF['Oct_22'], DF['Nov_22'], DF['Dic_22'], DF['Ene_23'], DF['Feb_23'], DF['Mar_23'], DF['Abr_23'], DF['May_23'], DF['Jun_23'], DF['Jul_23'], DF['Ago_23'], DF['Sep_23'], DF['Oct_23'], DF['Nov_23'], DF['Dic_23']]
tablas_mensuales = pd.concat(dfs_mensuales, ignore_index = True)

# Creamos 2 DataFrames
# Información de clientes:
info_clientes = tablas_mensuales.drop_duplicates(subset = ['IdPersona'])
info_clientes = info_clientes[['IdPersona', 'Edad', 'Sexo', 'EsAbonado']].copy()

# Se eliminan las filas que presentan valores de edad incorrectos
info_clientes['Edad'] = pd.to_numeric(info_clientes['Edad'], errors='coerce')
info_clientes = info_clientes[info_clientes['Edad'].notna()]

# Información variable en el tiempo (altas, bajas, tipo de abono):
Info_fechas = tablas_mensuales.drop_duplicates(subset = ['IdPersona', 'FechaAlta', 'FechaBajaAbono'])
altas_bajas_abonos = Info_fechas[['IdPersona', 'FechaAlta', 'FechaBajaAbono', 'BajaPorCambio', 'TipoAbono']].copy()

In [41]:
# Se crea una lista con los ID de usuarios que tienen el abono tipo: ACCÉS 1 MES (para usar más adelante)
Ids_Abono_1mes = altas_bajas_abonos[altas_bajas_abonos['TipoAbono'] == 'ACCÉS 1 MES']['IdPersona'].tolist()

# Tabla Altas-Bajas-Accesos

In [42]:
# Convertimos las columnas FechaAlta y FechaBajaAbono a tipo fecha para poder operar con la misma
altas_bajas_abonos['FechaBajaAbono'] = pd.to_datetime(altas_bajas_abonos['FechaBajaAbono'], format='%d/%m/%Y', errors = 'coerce')
altas_bajas_abonos['FechaAlta'] = pd.to_datetime(altas_bajas_abonos['FechaAlta'], format='%d/%m/%Y', errors = 'coerce')
# altas_bajas['FechaBajaAbono'] = altas_bajas['FechaBajaAbono'].dt.date() # (Supuestamente para convertir DateTime to Date, pero no funciona con series) - No se encuengtra la manera de pasar a Date

# Convertimos la columna BajaPorCambio en booleana
altas_bajas_abonos['BajaPorCambio'] = altas_bajas_abonos['BajaPorCambio'].map({'Sí': True, 'No': False})

In [43]:
# Rellenamos las fechas vacías de FechasBaja Abono (usuarios que no se dan de baja) con una fecha futura para poder operar más adelante
Fecha_sin_baja = pd.to_datetime('2024-01-02')
altas_bajas_abonos['FechaBajaAbono'] = altas_bajas_abonos['FechaBajaAbono'].fillna(Fecha_sin_baja)

In [44]:
# altas_bajas_abonos = altas_bajas_abonos[altas_bajas_abonos['IdPersona'].isin(usuarios)]
# altas_bajas_abonos.drop_duplicates(inplace= True)

In [45]:
# Juntamos ambas tablas en la variable IdPersona para que se repitan todas esas filas que para un mismo ID presentan variedad de datos en altas, bajas y abonos
accesos_altas_bajas = pd.merge(accesos, altas_bajas_abonos, on='IdPersona', how='inner')
accesos_altas_bajas.drop_duplicates(inplace= True)

# Unión de accesos con altas-bajas-accesos y limpieza de datos

In [46]:
# Creamos una columna booleana que indica para cada mes de accesos, si el usuario esta dado de alta o no (si la FechaAlta es igual o inferior al último día del mes y la FechaBajaAbono es superior al primer día del mes)
accesos_altas_bajas['EsSocio'] = ((accesos_altas_bajas['Mes'] + pd.DateOffset(months=1) - pd.DateOffset(days=1)) >= accesos_altas_bajas['FechaAlta']) & (accesos_altas_bajas['Mes'] < accesos_altas_bajas['FechaBajaAbono'])

# Descartamos todas las filas en que los usuarios no están dados de alta
accesos_altas_bajas = accesos_altas_bajas[accesos_altas_bajas['EsSocio']].copy()

# Eliminamos datos "duplicados" - Si un cliente se da de baja y de alta el mismo més el registro se duplica (con FechaAlta y/o FechaBaja diferente)
accesos_altas_bajas.drop_duplicates(subset = ['IdPersona', 'Mes'],  keep='first', inplace = True)

In [47]:
# Separamos los usuarios en 2 DF, uno para los que tienen accesos 'ACCÉS 1 MES' y el resto de usuarios
accesos_altas_bajas_abono_1mes = accesos_altas_bajas[accesos_altas_bajas['IdPersona'].isin(Ids_Abono_1mes)]
accesos_altas_bajas_resto_abonos = accesos_altas_bajas[~accesos_altas_bajas['IdPersona'].isin(Ids_Abono_1mes)]

In [48]:
# Se crea una nueva columna booleana que indica la baja en el mes que se da de baja
# Para la mayoría de usuarios se categoriza la baja por rango de fechas
accesos_altas_bajas_resto_abonos['Baja'] = (accesos_altas_bajas_resto_abonos['Mes'] < accesos_altas_bajas_resto_abonos['FechaBajaAbono']) & (accesos_altas_bajas_resto_abonos['FechaBajaAbono'] <= (accesos_altas_bajas_resto_abonos['Mes'] + pd.DateOffset(months=1))) & (accesos_altas_bajas_resto_abonos['BajaPorCambio'] == False)

In [49]:
# Se crea una nueva columna booleana que indica la baja en el mes que se da de baja
# Para los usuarios ACCÉS 1 MES se emplea el mismo concepto de baja, pero añadiendo que si el mes siguiente se vuelve a dar de alta no se contemple como baja.

# Reseteamos el índice para poder recorrer y escribir correctamente en el DataFrame
accesos_altas_bajas_abono_1mes.reset_index(drop=True, inplace= True)

# Bucle para recorrer el DataFrame, creando una variable que sea el registro actual y otra que sea el registro siguiente, con un IF para el último registro (no tiene la variable 'siguiente_registro')
for indice in range(len(accesos_altas_bajas_abono_1mes)):
    registro_actual = accesos_altas_bajas_abono_1mes.iloc[indice]
    if indice + 1 < len(accesos_altas_bajas_abono_1mes):
        registro_siguiente = accesos_altas_bajas_abono_1mes.iloc[indice + 1]

        if (registro_actual['IdPersona'] == registro_siguiente['IdPersona']) and (registro_actual['Mes'] + pd.DateOffset(months=1) == registro_siguiente['Mes']):
            accesos_altas_bajas_abono_1mes.at[indice, 'Baja'] = False
        else:
            if (registro_actual['Mes'] < registro_actual['FechaBajaAbono']) & (registro_actual['FechaBajaAbono'] <= (registro_siguiente['Mes'])) & (registro_actual['BajaPorCambio'] == False):
                accesos_altas_bajas_abono_1mes.at[indice, 'Baja'] = True
            else:
                accesos_altas_bajas_abono_1mes.at[indice, 'Baja'] = False
    else:
        if (registro_actual['Mes'] < registro_actual['FechaBajaAbono']) & (registro_actual['FechaBajaAbono'] <= (registro_actual['Mes'] + pd.DateOffset(months=1))) & (registro_actual['BajaPorCambio'] == False):
            accesos_altas_bajas_abono_1mes.at[indice, 'Baja'] = True
        else:
            accesos_altas_bajas_abono_1mes.at[indice, 'Baja'] = False

In [50]:
# Se juntan ambos DataFrames
accesos_altas_bajas = pd.concat([accesos_altas_bajas_abono_1mes, accesos_altas_bajas_resto_abonos], axis=0)
accesos_altas_bajas = accesos_altas_bajas.sort_values(by=['IdPersona', 'Mes'], ascending=True)


# Cálculo de antiguedad de los miembros

In [51]:
# Se crea un nuevo DataFrame con el primer registro de cada usuario
IdPersonas = accesos_altas_bajas.groupby('IdPersona').first().reset_index()

# Se calcula la antiguedad inicial de cada miembro
def Antiguedad_Inicial(fila):
    Diferencia = relativedelta(fila['Mes'], fila['FechaAlta'])
    return Diferencia.years * 12 + Diferencia.months

IdPersonas['AntiguedadInicial'] = IdPersonas.apply(Antiguedad_Inicial, axis=1)

In [52]:
# Unimos el DataFrame princiapl con el de la antiguedad inicial de cada usuario
accesos_altas_bajas = accesos_altas_bajas.merge(IdPersonas[['IdPersona', 'AntiguedadInicial']], on='IdPersona', how='inner')

In [53]:
# Se calcula la antiguedad para cada mes partiendo de la antiguedad inicial previamente calculada, reiniciando en caso de baja
def Antiguedad_Miembro(Usuario):
    Antiguedad = Usuario['AntiguedadInicial'].iloc[0]
    Lista_Antiguedades = []
    for Baja in Usuario['Baja']:
        Lista_Antiguedades.append(Antiguedad)
        if Baja:
            Antiguedad = 0
        else:
            Antiguedad = Antiguedad + 1
    Usuario['MesesAntiguedad'] = Lista_Antiguedades
    return Usuario

accesos_altas_bajas = accesos_altas_bajas.groupby('IdPersona').apply(Antiguedad_Miembro).reset_index(drop=True)

C:\Users\Joan\AppData\Local\Temp\ipykernel_3312\4217161863.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  accesos_altas_bajas = accesos_altas_bajas.groupby('IdPersona').apply(Antiguedad_Miembro).reset_index(drop=True)


# Mes de Alta

In [54]:
# Creamos una nueva columna booleana con las bajas desfasadas 1 registro y realizamos la suma acumulativas de dichos valores por cada usuario
accesos_altas_bajas['Realta'] = accesos_altas_bajas.groupby('IdPersona')['Baja'].shift(1).fillna(False).astype(bool)
accesos_altas_bajas['Realta'] = accesos_altas_bajas.groupby('IdPersona')['Realta'].cumsum()

# Se agrupan en conjuntos de usuarios y valor acumulado para diferenciar los periodos de los diferentes abonos y obtener dicho valor mensual
accesos_altas_bajas['FechaAltaPorGrupoAbono'] = accesos_altas_bajas.groupby(['IdPersona', 'Realta'])['FechaAlta'].transform('first')
accesos_altas_bajas['MesAlta'] = accesos_altas_bajas['FechaAltaPorGrupoAbono'].dt.month

C:\Users\Joan\AppData\Local\Temp\ipykernel_3312\3142005160.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  accesos_altas_bajas['Realta'] = accesos_altas_bajas.groupby('IdPersona')['Baja'].shift(1).fillna(False).astype(bool)


# Mes Actual

In [55]:
# Aseguramos que la columna 'Mes' está en formato fecha
accesos_altas_bajas['Mes'] = pd.to_datetime(accesos_altas_bajas['Mes'])

# Agregamos una columna que determine el mes actual, últil para conocer el mes de la baja
accesos_altas_bajas['MesActual'] = accesos_altas_bajas['Mes'].dt.month

# Cálculo de accesos trimestrales

In [56]:
# Se realiza el sumatorio de los 3 meses de accesos por registro
accesos_altas_bajas['AccesosTrimestrales'] = accesos_altas_bajas['Accesos'] + accesos_altas_bajas['Accesos-1'] + accesos_altas_bajas['Accesos-2']

# Unión con la información de los usuarios

In [57]:
# Se une la tabla principal con la tabla de información de usuario
Tabla_semicompleta = pd.merge(accesos_altas_bajas, info_clientes, on='IdPersona', how='inner')

# Limpieza de las columnas sobrantes

In [58]:
Tabla_completa = Tabla_semicompleta.drop(columns=['FechaAlta', 'FechaBajaAbono', 'BajaPorCambio', 'EsSocio', 'AntiguedadInicial', 'Realta', 'FechaAltaPorGrupoAbono'])

In [59]:
Tabla_completa = Tabla_completa[['IdPersona', 'Mes', 'Edad', 'Sexo', 'EsAbonado', 'TipoAbono', 'MesAlta', 'MesActual', 'Accesos', 'Accesos-1', 'Accesos-2', 'AccesosTrimestrales', 'MesesAntiguedad', 'Baja']]

In [60]:
# Tabla_completa

# Conversión de las variables categóricas

In [61]:
# Asociamos la tabla a una variable diferentes
Tabla_modelos = Tabla_completa

In [62]:
# Se cambia el nombre de la columna 'Sexo'
Tabla_modelos.rename(columns={'Sexo': 'Genero_masculino'}, inplace=True)

# Cambiamos los valores str a booleanos
Tabla_modelos['Genero_masculino'] = Tabla_modelos['Genero_masculino'].map({'Hombre': True, 'Mujer': False})
Tabla_modelos['EsAbonado'] = Tabla_modelos['EsAbonado'].map({'Sí': True, 'No': False})

In [63]:
# Usamos get_dummies para la columna TipoAbono y el mes de alta
Tabla_modelos = pd.get_dummies(Tabla_modelos, columns = ['TipoAbono', 'MesAlta', 'MesActual'])

In [64]:
# Tabla_modelos

# Exportamos el DataFrame a Excel

In [65]:
ruta_archivo = 'C:/Información/Master Big Data n Analytics/TFM/GitHub/Chrun-Model/DatosParaModelo.xlsx'

In [67]:
Tabla_modelos.to_excel(ruta_archivo, index=False, sheet_name='DataFrame')